В данной статье авторы описывают ряд изменений в алгоритме обучения языковой модели [BERT]($GPT и BERT$). Архитектурно модель RoBERTa, как и BERT, ничем не отличается от энкодера, предложенного в оригинальной работе по трансформерам ([Vaswani et al. 2017]($Attention Is All You Need$)).

Применив описанные в работе походы, авторы сравнивают качество полученной языковой модели RoBERTa с созданными ранее BERT и [XLNet]($XLNet: Generalized Autoregressive Pretraining for Language Understanding$). Сравнение происходит путем файн-тюнинга на различных задачах и сравнения метрики качества. Впрочем, поскольку задач и датасетов существует много, сравнение получается не всегда однозначным: одна модель может показывать лучший результат на одних задачах, другая модель - на других задачах.

Отличия в алгоритме обучения RoBERTa перечислены ниже.

**Отказ от использования задачи next sentence prediction (NSP) в обучении**

Согласно авторам BERT, добавление задачи NSP к задаче masked language model (MLM) повышает качество, однако у авторов данной работы получается иной результат. Авторы предполагают, что при обучении BERT сравнивалось только лишь наличие или отсутствие NSP loss, тогда как формат входных данных всегда соответствовал NSP, то есть 2 фрагмента текста с разделителем между ними. Отказ от такого формата (и от NSP loss вместе с ним) повышает качество.

**Full-sentences vs Doc-sentences**

В подходе full-sentences на вход модель принимает батч, каждый пример в котором состоит из 512 токенов. Если некий текст имеет длину меньше 512, то текст дополняется до 512 токенов другим текстом, отделенного от предыдущего текста сепаратором [SEP]. Этот формат близок к формату, используемому в NSP, однако здесь это делается лишь по необходимости. В альтернативном подходе doc-sentences пример в батче не может быть склейкой разных текстов. В этом случае примеры могут получиться короче. По видимому, авторы предлагают собирать из более коротких примеров батчи большего размера.

>  Inputs sampled near the end of a document may be shorter than 512 tokens, so we dynamically increase the batch size in these cases to achieve a similar number of total tokens as FULLSENTENCES.

Подход doc-sentences дает результат немного лучше, однако в дальнейшем авторы все же используют full-sentences, чтобы не требовалось динамически менять размер батча.

**Обучение с большим размером батча**

Авторы обнаруживают, что увеличение размера батча в N раз и сокращение количества шагов в N раз позволяет повысить качество, если при этом также повышать learning rate. Результаты обобщены в таблице, приведенной ниже. Остается надеяться, что learning rate подобран по всех случаях оптимально.

<img src="assets/roberta.jpg" width="350" align="center">

Конечно, увеличение размера батча также может увеличить эффективность обучения благодаря параллелизации. Даже при обучении на одном GPU большой размер батча можно эмулировать с помощью нескольких прямых проходов и накопления градиента, что также даст прирост скорости обучения.

Увеличение размера батча в N раз при сокращении количества шагов в N раз и увеличении learning rate в N раз эквивалентно накоплению градиента за N шагов, и затем применения накопленного градиента для обновления весов, вместо применения градиента после каждого шага. Однако не совсем понятно почему такое изменение дает положительное влияние на качество. Как мне кажется, этот вопрос требует дальнейшего исследования.

**Динамическое маскирование в задаче masked language model.**

В BERT для каждого текста в датасете создавалось по 10 разных масок. Модель BERT обучалась в течение 40 эпох, то есть каждая маска использовалась 4 раза. В RoBERTa каждую эпоху создаются новые маски. Отличия в качестве получаются не слишком существенные. Скорее это делается для того, чтобы сократить объем требуемого места на диске.

> We generate the masking pattern every time we feed a sequence to the model. This becomes crucial when pretraining for more steps or with larger datasets.

**Изменение параметров оптимизатора**

В процессе подбора гиперпараметров было обнаружено, что параметры epsilon и $\beta_2$ в оптимизаторe [Adam]($Adam: A Method for Stochastic Optimization$) оказывает существенное влияние на процесс обучения.

> We additionally found training to be very sensitive to the Adam epsilon term, and in some cases we obtained better performance or improved stability after tuning it. Similarly, we found setting $\beta_2 = 0.98$ to improve stability when training with large batch sizes.

**Изменение способа токенизации**

В BERT используется способ токенизации [WordPiece]($Googles Neural Machine Translation System: Bridging the Gap between Human and Machine Translation$), который авторы данной работы иначе называют BPE (разница между WordPiece и BPE [минимальна](https://datascience.stackexchange.com/questions/86572/bert-uses-wordpiece-roberta-uses-bpe)). В RoBERTa при инициализации BPE используются не отдельные символы unicode, а отдельные байты. Это позволяет избежать ситуации когда существенная часть словаря заполнена различными редкими символами unicode. По-видимому, необходимость в таком шаге возникла из-за расширения объема и разнообразия обучающих данных на различных языках, о котором пойдет речь дальше.

> Unicode characters can account for a sizeable portion of this vocabulary when modeling large and diverse corpora, such as the ones considered in this work.

**Дополнительные данные для обучения**

Оценив влияние каждого из предложенных изменений на качество обучения (при том же объеме обучающих данных), авторы делают следующий шаг: собирают больший датасет для обучения и обучают модель большее число шагов, достигая еще более высокого качества.

BERT обучалась на следующих данных:

- Английская Википедия
- BookCorpus

В сумме эти два источника давали примерно 13-16 Гб текста (в зависимости от способа очистки текста). Авторы данной работы добавляют еще три источника данных:

- Английская часть CommonCrawl News - 76 Гб
- OpenWebText (тексты, извлеченные со страниц, ссылки на которые была размещены на Reddit) - 38 Гб
- Датасет Stories (подмножество CommonCrawl data) - 31 Гб

В сводной таблице приводится сравнение результатов языковых моделей на трех датасетах. Строка "with Books + Wiki" означает обучение на тех же данных, на которых обучалась модель BERT. В целом качество модели RoBERTa сопоставимо с [XLNet]($XLNet: Generalized Autoregressive Pretraining for Language Understanding$) и превосходит качество оригинальной версии BERT. И если XLNet является существенной модификацией BERT, созданной с целью повысить качество, то RoBERTa достигает аналогичного качества лишь с небольшими изменениями относительно BERT, описанными выше.

<img src="assets/roberta2.jpg" width="600" align="center">